# v.dissolve

This notebook presents couple examples of _v.dissolve_ and examination of its outputs.

## Setup

We will be using the NC SPM sample location.

In [ ]:
import json
import subprocess
import sys

# Ask GRASS GIS where its Python packages are.
sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)

# Import GRASS packages
import grass.script as gs
import grass.jupyter as gj

# Start GRASS Session
gj.init("~/data/grassdata/nc_basic_spm_grass7/user1")

## Dissolve by Attribute

We will use ZIP codes to create town boundaries by dissolving boundaries of ZIP code areas. Let's see the ZIP codes:

In [ ]:
zipcodes = "zipcodes"
town_map = gj.Map()
town_map.d_vect(map=zipcodes)
town_map.show()

 We dissolve boudaries between ZIP codes which have the same town name which is in the NAME attribute.

In [ ]:
towns = "towns_from_zipcodes"
gs.run_command(
    "v.dissolve",
    input=zipcodes,
    column="NAME",
    output=towns,
)

Color boudaries according to the primary key column called cat and display.

In [ ]:
gs.run_command("v.colors", map=towns, use="attr", column="cat", color="wave")

In [ ]:
town_map.d_vect(map=towns)
town_map.show()

In [ ]:
town_map.d_vect(map=zipcodes, fill_color="none")
town_map.show()

## Dissolve with Attribute Aggregation

Now let's count number of ZIP codes in each town and compute total area as a sum of an existing column in the dataset.

In [ ]:
towns_with_area = "towns_with_area"
gs.run_command(
    "v.dissolve",
    input=zipcodes,
    column="NAME",
    output=towns_with_area,
    aggregate_column="SHAPE_Area,SHAPE_Area",
    aggregate_method="count,sum",
    result_column="num_zip_codes,town_area",
)

Print the computed attributes:

In [ ]:
table = json.loads(gs.read_command("v.db.select", map=towns_with_area, format="json"))

In [ ]:
for row in table["records"]:
    print(f'{row["NAME"]:<14} {row["num_zip_codes"]:>2} {row["town_area"]:>12.0f}')

Now color the result using the total area:

In [ ]:
gs.run_command(
    "v.colors", map=towns_with_area, use="attr", column="town_area", color="plasma"
)

In [ ]:
town_map = gj.Map()
town_map.d_vect(map=towns_with_area)
town_map.show()

## Images for Documentation

Here, we use some of the data created above to create images for documentation.

In [ ]:
zip_map = gj.Map()
zip_map.d_vect(map=towns, flags="s")
zip_map.d_vect(map=zipcodes, color="#222222", width=2, type="boundary")
zip_map.d_legend_vect()
zip_map.show()

In [ ]:
town_map = gj.Map()
town_map.d_vect(map=towns, flags="s")
town_map.d_vect(map=towns_with_area, color="#222222", width=2, type="boundary")
town_map.d_legend_vect()
town_map.show()

In [ ]:
# This cell requires pngquant and optipng.
zip_map.save("v_dissolve_zipcodes.png")
town_map.save("v_dissolve_towns.png")
for filename in ["v_dissolve_zipcodes.png", "v_dissolve_towns.png"]:
    !pngquant --ext ".png" -f {filename}
    !optipng -o7 {filename}

## Test

For a small dataset, we can easily compute the same attribute values in Python. We do this assuming that all areas (polygons) with same value will be dissolved (merged) together possibly creating multipolygons.

In [ ]:
from collections import defaultdict

# Get the original attribute data.
zip_table = json.loads(gs.read_command("v.db.select", map=zipcodes, format="json"))
# Restructure original data for easy lookup of area.
zip_records_by_town = defaultdict(list)
for row in zip_table["records"]:
    zip_records_by_town[row["NAME"]].append(row["SHAPE_Area"])

# Check each row in the original table.
for row in table["records"]:
    town_name = row["NAME"]
    town_area = row["town_area"]
    town_zip_codes = row["num_zip_codes"]
    areas_by_zip = zip_records_by_town[town_name]
    # Check number ZIP codes.
    if len(areas_by_zip) != town_zip_codes:
        raise RuntimeError(f'Incorrect number of zipcodes in town {row["NAME"]}')
    # Check total area.
    if round(sum(areas_by_zip)) != round(town_area):
        raise RuntimeError(
            f'Incorrect area for {row["NAME"]}: {sum(areas_by_zip)} != {town_area}'
        )
print("No exceptions. Test passed.")